In [1]:
import requests
from bs4 import BeautifulSoup
import selenium
import pandas
from selenium import webdriver

In [2]:
options = webdriver.ChromeOptions()
options.add_argument("--ignore-certificate-errors")
options.add_argument("--incognito")
#options.add_argument('--headless')
options.add_experimental_option('excludeSwitches', ['enable-logging'])
driver = webdriver.Chrome(options=options)
driver.implicitly_wait(100)

In [3]:
url = "https://www.glassdoor.ie/Reviews/Aptiv-Reviews-E1891089.htm?filter.iso3Language=eng"
driver.get(url)

In [23]:
html_soup = BeautifulSoup(driver.page_source, "html.parser")
containers = html_soup.find_all("div", class_="gdReview")
for container in containers:
    rating = container.find('span', class_='ratingNumber mr-xsm')
    rating = rating.text
    print('star reviews:', rating)

    employee_Status = container.find('span', class_='pt-xsm pt-md-0 css-1qxtz39 eg4psks0')
    employee_Status = employee_Status.text
    print('employee_status:', employee_Status)

    # review_title = container.find('a', class_='mb-xxsm')
    # review_title = container.text
    # print(review_title)

    # list=[]
    # for i in container:
    #     list.append(i.get_text())
    # print(list)

    
    #print(html_soup.select('h2.mb-xxsm')[0].get_text())
    # for i,j in zip(html_soup.select('h2.mb-xxsm'),range(len(html_soup.select('h2.mb-xxsm')))):
    #     print(html_soup.select('h2.mb-xxsm')[j].get_text())

    title = container.find('a', class_="reviewLink")
    title = title.text
    print('review title:', title)

    date_posted = container.find('span', class_='authorJobTitle middle common__EiReviewDetailsStyle__newGrey')
    date_posted = date_posted.text
    print('date & position: ', date_posted)

    author_location = container.find('span', class_='authorLocation')
    author_location = author_location.text
    print('author location:', author_location)
    
    pros = container.find('p', class_='mb-0 strong')
    pros = pros.text
    print(pros)


    # job_title = container.find('span', class_='authorJobTitle middle common__EiReviewDetailsStyle__newGrey')
    # job_title = job_title.text
    # print(job_title)

    # pros = container.find('p', class_='mb-0 strong')
    # pros = pros.text
    # print(job_title)

    print()


star reviews: 4.0
employee_status: Current Employee
review title: Great Culture and Company That Cares
date & position:  13 Jan 2021 - Senior Corporate Recruiter
author location: Troy, MI
Pros
Not much politics and very few ego's. Everyone is willing to help which is refreshing. Not very tough recruiting A Player Talent as Aptiv is a place where most want to work.

star reviews: 5.0
employee_status: Current Employee, more than 1 year
review title: Shaping the Future of Mobility!
date & position:  14 Apr 2021 - Program Manager
author location: Dublin, Dublin
Pros


AttributeError: 'NoneType' object has no attribute 'text'